# Sample Demand Profile

> US Domestic Hot Water 1 minute Draw Profiles

https://github.com/jmaguire1/WaterHeaterPythonModel/tree/master/data_files/DrawProfiles

In [1]:
#| default_exp demand

In [ ]:
#| hide
from nbdev.showdoc import *

In [1]:
#| export
import pandas as pd	
import numpy as np
import random

In [2]:
#| export
def load_demand(bed=1,unit=1):
    if not (bed in [1, 2, 3, 4, 5]) and (unit in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]):
        print('Not valid bed / unit choice')
        return pd.DataFrame()
    base = pd.read_csv('../data/drawprofiles/MinuteDrawProfilesMaxFlows.csv')
    df = pd.read_csv(f'../data/drawprofiles/DHWDrawSchedule_{bed}bed_unit{unit}_1min_fraction.csv')
    df = df.fillna(0)
    '''
    Translate between the BEOpt Data
    see https://github.com/jmaguire1/WaterHeaterPythonModel/blob/01f8a3c88096c9cbdbb49c354405d65fb37c2d7e/WH_control_fleet.py
    '''
    sh_hsp_tot = 14.0 + 4.67 * bed
    s_hsp_tot = 12.5 + 4.16 * bed
    cw_hsp_tot = 2.35 + 0.78 * bed
    dw_hsp_tot = 2.26 + 0.75 * bed
    b_hsp_tot = 3.50 + 1.17 * bed
    df.Showers =  base.loc[((base.Bedrooms == bed) & (base.Unit== unit)),'Showers Max'].values[0] * df.Showers * \
                    sh_hsp_tot / base.loc[((base.Bedrooms == bed) & (base.Unit== unit)),'Showers Sum'].values[0]
    df.Sinks =  base.loc[((base.Bedrooms == bed) & (base.Unit== unit)),'Sinks Max'].values[0] * df.Sinks * \
                    s_hsp_tot / base.loc[((base.Bedrooms == bed) & (base.Unit== unit)),'Sinks Sum'].values[0]
    df.CW =  base.loc[((base.Bedrooms == bed) & (base.Unit== unit)),'CW Max'].values[0] * df.CW * \
                    cw_hsp_tot / base.loc[((base.Bedrooms == bed) & (base.Unit== unit)),'CW Sum'].values[0]
    df.DW =  base.loc[((base.Bedrooms == bed) & (base.Unit== unit)),'DW Max'].values[0] * df.DW * \
                    dw_hsp_tot / base.loc[((base.Bedrooms == bed) & (base.Unit== unit)),'DW Sum'].values[0]
    df.Baths =  base.loc[((base.Bedrooms == bed) & (base.Unit== unit)),'Baths Max'].values[0] * df.Baths * \
                    b_hsp_tot / base.loc[((base.Bedrooms == bed) & (base.Unit== unit)),'Baths Sum'].values[0]
    # The flows can be split into hot and warm if more granular detail is required
    df = df * 3.785411784 # convert from US GPM to lpm
    df['total_lpm'] = df.sum(axis=1)
    df['total_m3s'] = df.total_lpm/60000 * 45 / 60
    df[f'{bed}_{unit}'] = df.total_m3s
    df['timestamp'] = pd.date_range(start='1/1/2020', periods=len(df), freq='60S')
    df = df.set_index('timestamp')
    

    return df[f'{bed}_{unit}']

In [5]:
#| export
# flow = pd.concat([load_demand(bed=b,unit=u) for u in range(10) for b in range(1,6)], axis=1)

: 

: 

In [4]:
flow= load_demand(bed=2,unit=3)
flow

timestamp
2020-01-01 00:00:00    0.0
2020-01-01 00:01:00    0.0
2020-01-01 00:02:00    0.0
2020-01-01 00:03:00    0.0
2020-01-01 00:04:00    0.0
                      ... 
2020-12-30 23:55:00    0.0
2020-12-30 23:56:00    0.0
2020-12-30 23:57:00    0.0
2020-12-30 23:58:00    0.0
2020-12-30 23:59:00    0.0
Name: 2_3, Length: 525600, dtype: float64

Other potential demand profiles
https://uknowledge.uky.edu/cgi/viewcontent.cgi?article=1082&context=bae_etds

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()